In [ ]:
get_ipython().magic('matplotlib inline')
get_ipython().magic('reload_ext autoreload')
get_ipython().magic('autoreload 2')
from anchor_detect import *
from anchor_check import *

# Check anchor distribution

In [45]:
meta_dict = pickle.load(open('../data/video_meta_dict.pkl', 'rb'))
com_dict = pickle.load(open('../data/commercial_dict.pkl', 'rb'))
anchor_dict = pickle.load(open('../data/anchor_dict.pkl', 'rb'))

In [ ]:
video = 'CNNW_20160721_160000_Inside_Politics'
if video in com_dict:
    com_list = com_dict[video]
else:
    com_list = None 
plot_distribution(meta_dict[video], anchor_dict[video], com_list)

# Train anchor classifier on the same type of show

In [2]:
anchor_dict = pickle.load(open('../data/anchor_dict_half0.pkl', 'rb'))

In [ ]:
# prepare training examples
show_name = 'Hannity'
# def train_identity_classifier(anchor_dict, show_name):
FACE_POS_THRESH = 1.0
FACE_NEG_THRESH = 1.1
anchor_dict_show = group_by_show(anchor_dict)
real_anchor = cluster_real_anchor(anchor_dict_show[show_name])
anchor_center = build_anchor_center(anchor_dict_show[show_name])

identity = real_anchor[0]
data = []
meta = []
labels = []
positive = 0
## collect positive examples
for video, anchor_group in anchor_dict_show[show_name].items():
    for anchor_person in anchor_group:
        for p in anchor_person:
            if np.linalg.norm(p['feature'] - identity) < FACE_POS_THRESH:
                data.append(p['feature'])
                meta.append({'video': video, 'fid': p['fid'], 'bbox': p['bbox']})
                labels.append(1)
                positive += 1
print("positive examples: %d" % positive)
## collect negative examples
negative = 0
for show, show_dict in anchor_dict_show.items():
#     if negative > positive:
#             break
    if show == show_name:
        continue
    other_center = build_anchor_center(show_dict)
    for video, centers in other_center.items():
        for p in centers:
            if np.linalg.norm(p['feature'] - identity) > FACE_NEG_THRESH:
                data.append(p['feature'])
                meta.append({'video': video, 'fid': p['fid'], 'bbox': p['bbox']})
                labels.append(-1)
                negative += 1
#         break
print("negative examples: %d" % negative)


In [37]:
## split dataset
data_train, meta_train, labels_train, data_test, meta_test, labels_test = split_dataset(data, meta, labels) 

In [ ]:
from sklearn.svm import SVC 

## train
# clf = SVC(probability=True)
clf = SVC()
clf.fit(data_train, labels_train) 

In [ ]:
# labels_predict = clf.predict(data_test)
score = clf.score(data_test, labels_test)
print(score)
# proba = clf.predict_proba(data_test)

In [ ]:
weak_pos = []
weak_neg = []
PROB_THRESH = 0.01
for idx, p in enumerate(proba):
    if p[0] < p[1] and p[0] > PROB_THRESH:
        print('pos', p)
        weak_pos.append(meta_test[idx])
    if p[0] > p[1] and p[1] > PROB_THRESH:
        print('neg', p)
        weak_neg.append(meta_test[idx])
print("Num of weak positive: %d" % len(weak_pos))
print("Num of weak negative: %d" % len(weak_neg))
weak_dict = {'weak_pos': weak_pos, 'weak_neg': weak_neg}

# Unsupervised check using the same type of show

In [ ]:
anchor_dict = pickle.load(open('../data/anchor_dict_half0.pkl', 'rb'))

anchor_dict_show = group_by_show(anchor_dict)

check_anchor_by_show(anchor_dict_show['Wolf'])